In [1]:
from gzeus import Chunker, stream_polars_csv_gz
import polars as pl
import pandas as pd

In [2]:
# The dataset is available at:
# https://catalog.data.gov/dataset/insurance-complaints-all-data

In [3]:
def chunk_load_data_pandas(path:str, chunk_size:int = 10_000) -> pd.DataFrame:
    frames = [
        df_chunk
        for df_chunk in pd.read_csv("../data/insurance.csv.gz", iterator = True, chunksize=chunk_size, engine="c")   
    ]
    return pd.concat(frames)

def chunk_load_data_gzeus2(path:str, max_bytes:int = 1_000_000) -> pl.DataFrame:
    # Turn portion of the produced bytes into a DataFrame. Only possible with Polars, 
    # or dataframe packages with "lazy" capabilities. Lazy read + filters ensure 
    # only necessary bytes are copied into our dataframe 
    def get_necessary_data(df:pl.LazyFrame) -> pl.DataFrame:
        return df.filter(
            (pl.col("Confirmed complaint") != 'No')
            & (pl.col("Keywords").is_not_null())
        ).select(
            'Complaint number',
            'Complaint filed against',
            'Complaint filed by',
            'Reason complaint filed',
            'Confirmed complaint',
            'Received date',
            'Closed date',
            'Complaint type',
            'Coverage level',
        ).collect()

    return pl.concat(stream_polars_csv_gz(path, buffer_size=max_bytes, func = get_necessary_data))

def chunk_load_data_pandas2(path:str, chunk_size:int = 10_000) -> pd.DataFrame:
    def get_necessary_data(df:pd.DataFrame) -> pd.DataFrame:
        return df.loc[
            (df['Confirmed complaint'] != 'No') & (~df['Keywords'].isna())
            , :
        ][[
            'Complaint number',
            'Complaint filed against',
            'Complaint filed by',
            'Reason complaint filed',
            'Confirmed complaint',
            'Received date',
            'Closed date',
            'Complaint type',
            'Coverage level',
        ]]

    frames = [
        get_necessary_data(df_chunk)
        for df_chunk in pd.read_csv("../data/insurance.csv.gz", iterator = True, chunksize=chunk_size, engine="c")   
    ]
    return pd.concat(frames)

In [4]:
df = pl.concat(stream_polars_csv_gz("../data/insurance.csv.gz"))
df.head()

Complaint number,Complaint filed against,Complaint filed by,Reason complaint filed,Confirmed complaint,How resolved,Received date,Closed date,Complaint type,Coverage type,Coverage level,Others involved,Respondent ID,Respondent Role,Respondent type,Complainant type,Keywords
i64,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,str
1,"""METROPOLITAN LIFE INSURANCE CO…","""Relative""","""Cust Service Claim Handling""","""No""","""Other""","""06/12/2012""","""07/25/2012""","""Life, Accident and Health""","""Life & Annuity""","""Individual Life""",null,13191,"""Ins Co - Licensed/Active""","""Organization""","""INDV""",null
2,"""AETNA LIFE INSURANCE COMPANY""","""Provider""","""Delays (Claims Handling)""","""No""","""Information Furnished""","""06/21/2012""","""08/01/2012""","""Life, Accident and Health""","""Accident and Health""","""Group A&H""","""Insured""",245,"""Ins Co - Licensed/Active""","""Organization""","""ORG""",null
3,"""BLUE CROSS AND BLUE SHIELD OF …","""Provider""","""Denial Of Claim""","""No""","""Other""","""06/11/2012""","""07/30/2012""","""Life, Accident and Health""","""Accident and Health""","""Group A&H""",null,10047,"""Ins Co - Licensed/Active""","""Organization""","""ORG""",null
4,"""BLUE CROSS AND BLUE SHIELD OF …","""Provider""","""Denial Of Claim""","""No""","""Other""","""06/28/2012""","""07/30/2012""","""Life, Accident and Health""","""Accident and Health""","""Group A&H""",null,10047,"""Ins Co - Licensed/Active""","""Organization""","""ORG""",null
5,"""CHARTER OAK FIRE INSURANCE COM…","""Insured""","""Unsatisfactory Settle/Offer""","""No""","""Question of Fact; Contract Lan…","""06/13/2012""","""07/17/2012""","""Property and Casualty""","""Automobile""","""Individual Private Pass""",null,2918,"""Ins Co - Licensed/Active""","""Organization""","""INDV""","""2012 NORTH TEXAS TORNADOES; AD…"


In [5]:
df_pd = chunk_load_data_pandas("../data/insurance.csv.gz")
df_pd.head()

,Complaint number,Complaint filed against,Complaint filed by,Reason complaint filed,Confirmed complaint,How resolved,Received date,Closed date,Complaint type,Coverage type,Coverage level,Others involved,Respondent ID,Respondent Role,Respondent type,Complainant type,Keywords
0,1,METROPOLITAN LIFE INSURANCE COMPANY,Relative,Cust Service Claim Handling,No,Other,06/12/2012,07/25/2012,"Life, Accident and Health",Life & Annuity,Individual Life,NaN,13191,Ins Co - Licensed/Active,Organization,INDV,NaN
1,2,AETNA LIFE INSURANCE COMPANY,Provider,Delays (Claims Handling),No,Information Furnished,06/21/2012,08/01/2012,"Life, Accident and Health",Accident and Health,Group A&H,Insured,245,Ins Co - Licensed/Active,Organization,ORG,NaN
2,3,"BLUE CROSS AND BLUE SHIELD OF TEXAS, A DIVISIO...",Provider,Denial Of Claim,No,Other,06/11/2012,07/30/2012,"Life, Accident and Health",Accident and Health,Group A&H,NaN,10047,Ins Co - Licensed/Active,Organization,ORG,NaN
3,4,"BLUE CROSS AND BLUE SHIELD OF TEXAS, A DIVISIO...",Provider,Denial Of Claim,No,Other,06/28/2012,07/30/2012,"Life, Accident and Health",Accident and Health,Group A&H,NaN,10047,Ins Co - Licensed/Active,Organization,ORG,NaN
4,5,"CHARTER OAK FIRE INSURANCE COMPANY, THE",Insured,Unsatisfactory Settle/Offer,No,Question of Fact; Contract Language/Legal Issue,06/13/2012,07/17/2012,Property and Casualty,Automobile,Individual Private Pass,NaN,2918,Ins Co - Licensed/Active,Organization,INDV,2012 NORTH TEXAS TORNADOES; ADJUSTER'S HANDLIN...


# Benchmark 1 - (GZeus + Polars) vs. Pandas 

No work per chunk.

Tuning pandas chunksize doesn't really help with performance at all.

In [6]:
%%timeit
df = pl.concat(stream_polars_csv_gz("../data/insurance.csv.gz"))

217 ms ± 9.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
df = chunk_load_data_pandas("../data/insurance.csv.gz")

434 ms ± 3.02 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
df = pl.concat(stream_polars_csv_gz("../data/insurance.csv.gz", buffer_size=5_000_000)) # bigger chunks, 5mb per chunk

91.9 ms ± 1.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%%timeit
df = chunk_load_data_pandas("../data/insurance.csv.gz", chunk_size=50_000)

427 ms ± 4.92 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
df = chunk_load_data_pandas("../data/insurance.csv.gz", chunk_size=100_000) # the whole df is 260k rows

426 ms ± 6.26 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Benchmark 2 - (GZeus + Polars) vs. Pandas 

Some work per chunk.

GZeus + Polars runs faster because the workload can be optimized by Polars and further speeds up the process. On the other hand, pandas shows no speed improvement because it will read the full chunk regardless of any work you do on the chunk.

In [11]:
%%timeit
df = chunk_load_data_gzeus2("../data/insurance.csv.gz")


191 ms ± 1.72 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%%timeit
df = chunk_load_data_pandas2("../data/insurance.csv.gz")

443 ms ± 4.06 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%%timeit
df = chunk_load_data_gzeus2("../data/insurance.csv.gz", max_bytes=5_000_000) # bigger chunks

76.4 ms ± 362 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
%%timeit
df = chunk_load_data_pandas2("../data/insurance.csv.gz", chunk_size=50_000)

439 ms ± 2.66 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%%timeit
df = chunk_load_data_pandas2("../data/insurance.csv.gz", chunk_size=100_000)

439 ms ± 10.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
